In [22]:
%matplotlib inline

In [19]:
from absa.config import DATA_PATHS
from absa.dataset import load_dataset
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import cross_validate
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline, make_union
from sklearn.svm import SVC
from sklearn.metrics import f1_score
import spacy
nlp = spacy.load('en')

In [21]:
train_ds_path = DATA_PATHS['asba.semeval16.raw.train']

df_train = load_dataset(train_ds_path)

df_train = df_train.loc[:, ['id', 'text', 'category', 'polarity']]

df_train = pd.DataFrame({
    'polarity': df_train.groupby(['id', 'text', 'category'])['polarity'].apply(list),
}).reset_index()

df_train.head()

KeyboardInterrupt: 

# Extract Features

In [9]:
def lemmatize(x):
    return ' '.join([token.lemma_ for token in nlp(x)])

In [10]:
pipeline = make_union(
    TfidfVectorizer(preprocessor=lemmatize, stop_words='english', ngram_range=(1, 3)),
)

x_train = pipeline.fit_transform(df_train.text).toarray()

x_train.shape

/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['PRON', 'make', 'whatev'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


(1708, 16880)

# Prepare Labels

In [11]:
mlb = MultiLabelBinarizer()

y_train = mlb.fit_transform(df_train.polarity)

y_train.shape

(1708, 12)

# Cross Validatation

In [12]:
scoring = ['precision_micro', 'recall_micro', 'f1_micro']

clf = OneVsRestClassifier(GaussianNB())

scores = cross_validate(clf, x_train, y_train, scoring=scoring, cv=3)

print('Micro Precision Score\t', np.average(scores['test_precision_micro']))
print('Micro Recall Score\t', np.average(scores['test_recall_micro']))
print('Micro F1 Score\t\t', np.average(scores['test_f1_micro']))

Micro Precision Score	 0.33871496136454965
Micro Recall Score	 0.5252014396890562
Micro F1 Score		 0.4117880780780941


# Train Model

In [13]:
clf.fit(x_train, y_train)

OneVsRestClassifier(estimator=GaussianNB(priors=None, var_smoothing=1e-09),
                    n_jobs=None)

# Evaluate Model

In [14]:
test_ds_path = DATA_PATHS['asba.semeval16.raw.test.gold']

df_test = load_dataset(test_ds_path)

df_test = df_test.loc[:, ['id', 'text', 'category', 'polarity']]

df_test = pd.DataFrame({
    'polarity': df_test.groupby(['id', 'text', 'category'])['polarity'].apply(list),
}).reset_index()

df_test.head()

,text,categories
id,,
en_BlueRibbonSushi_478218171:0,Yum!,[FOOD#QUALITY]
en_BlueRibbonSushi_478218171:1,Serves really good sushi.,[FOOD#QUALITY]
en_BlueRibbonSushi_478218171:2,Not the biggest portions but adequate.,[FOOD#STYLE_OPTIONS]
en_BlueRibbonSushi_478218171:3,Green Tea creme brulee is a must!,[FOOD#QUALITY]
en_BlueRibbonSushi_478218171:4,Don't leave the restaurant without it.,[FOOD#QUALITY]


In [15]:
# Testing
x_test = pipeline.transform(df_test.text).toarray()

y_pred = clf.predict(x_test)

y_pred.shape

(587, 12)

In [16]:
y_true = mlb.transform(df_test.polarity)

y_true.shape

(587, 12)

In [17]:
('f1_score', f1_score(y_true, y_pred, average='micro'))

('f1_score', 0.4005791505791506)